In [ ]:
#!/usr/bin/env python3
"""
批次下載中央氣象署 CODiS 測站資料
日期範圍：2016-04-29 ~ 2025-07-16（每日一次）
------------------------------------------------
執行前請先：
    pip install requests tqdm
"""

from __future__ import annotations

import datetime as dt
import json
import time
from pathlib import Path
from typing import Dict, Any, Iterator, List

import requests
from tqdm import tqdm


API_URL = "https://codis.cwa.gov.tw/api/station?"
HEADERS: Dict[str, str] = {
    "accept": "application/json, text/javascript, */*; q=0.01",
    "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
    "x-requested-with": "XMLHttpRequest",
}
STN_ID = "C0F9Y0"     # 測站 ID
STN_TYPE = "auto_C0"  # 測站類型
UTC_OFFSET = "+08:00"  # 台灣時區


def daterange(start: dt.date, stop: dt.date) -> Iterator[dt.date]:
    """Yield every calendar date from *start* (inclusive) to *stop* (inclusive)."""
    for n in range((stop - start).days + 1):
        yield start + dt.timedelta(days=n)


def build_payload(day: dt.date) -> Dict[str, str]:
    """根據日期產生 POST 表單資料。"""
    date_str = day.strftime("%Y-%m-%d")
    return {
        "date": f"{date_str}T00:00:00.000{UTC_OFFSET}",
        "type": "report_date",
        "stn_ID": STN_ID,
        "stn_type": STN_TYPE,
        "more": "",
        "start": f"{date_str}T00:00:00",
        "end": f"{date_str}T23:59:59",
        "item": "",
    }


def fetch_day(session: requests.Session, day: dt.date) -> Dict[str, Any]:
    """向 CODiS 伺服器請求單日資料，失敗時丟出例外。"""
    payload = build_payload(day)
    resp = session.post(API_URL, headers=HEADERS, data=payload, timeout=30, verify=False)
    resp.raise_for_status()
    return resp.json()  # type: ignore[return-value]


def save_results(results: List[Dict[str, Any]], outfile: Path) -> None:
    """將結果寫入 JSON 檔（以 UTF-8 保存）。"""
    outfile.write_text(json.dumps(results, ensure_ascii=False, indent=2), encoding="utf-8")


def main() -> None:
    start_date = dt.date(2016, 4, 29)
    end_date   = dt.date(2025, 7, 16)

    results: List[Dict[str, Any]] = []
    outfile = Path("codis_C0F9Y0_20160429_20250716.json")

    with requests.Session() as sess:
        for day in tqdm(daterange(start_date, end_date), total=(end_date - start_date).days + 1):
            try:
                daily_data = fetch_day(sess, day)
                daily_data["query_date"] = day.isoformat()  # 方便後續識別
                results.append(daily_data)
            except Exception as exc:
                print(f"[WARN] {day}: {exc}")

            time.sleep(0.2)  # 禮貌性暫停，避免對伺服器造成過大負載

    save_results(results, outfile)
    print(f"\n✔ 完成！共取得 {len(results)} 筆日資料 → {outfile}")


if __name__ == "__main__":
    main()
